In [1]:
import cobra
import sys
import pandas as pd
import os
from itertools import chain
# currently working with local medusa installation -- anytime changes are made to medusa, we need to run installation
# again within the virtualenv by running setup.py
#import medusa
from medusa.reconstruct.expand.expand import iterative_gapfill_from_binary_phenotypes
from cobra.flux_analysis import flux_variability_analysis
from cobra.flux_analysis.parsimonious import add_pfba

/home/greg/Envs/ssl_ensembles/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/greg/Envs/ssl_ensembles/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/greg/Envs/ssl_ensembles/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/greg/Envs/ssl_ensembles/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
biolog_base_composition = pd.read_csv('../data/biolog_base_composition.csv',sep=',')
biolog_base_dict = dict(zip(biolog_base_composition['ID'],\
                          [1000 for i in range(0,len(biolog_base_composition['ID']))]))
biolog_thresholded = pd.read_csv('../data/plata_thresholded.csv',sep='\t',index_col=0)
biolog_base_dict

{'cpd00001_e': 1000,
 'cpd00007_e': 1000,
 'cpd00009_e': 1000,
 'cpd00011_e': 1000,
 'cpd00013_e': 1000,
 'cpd00030_e': 1000,
 'cpd00034_e': 1000,
 'cpd00048_e': 1000,
 'cpd00058_e': 1000,
 'cpd00063_e': 1000,
 'cpd00067_e': 1000,
 'cpd00099_e': 1000,
 'cpd00149_e': 1000,
 'cpd00205_e': 1000,
 'cpd00254_e': 1000,
 'cpd00971_e': 1000,
 'cpd10515_e': 1000,
 'cpd10516_e': 1000}

In [3]:
universal = cobra.io.load_json_model('../data/universal_mundy.json')

In [4]:
testmod = cobra.io.load_json_model('../data/modelseed_models/Staphylococcus aureus.json')

In [5]:
testmod.optimize()
testmod_removed = testmod.copy()
testmod_removed.remove_reactions(testmod_removed.reactions[0:20])
testmod_removed.optimize()

,fluxes,reduced_costs
rxn05667_c,0.0,-0.000000e+00
rxn00322_c,0.0,0.000000e+00
rxn05457_c,0.0,-0.000000e+00
rxn04271_c,0.0,-0.000000e+00
rxn13783_c,0.0,0.000000e+00
...,...,...
EX_cpd00066_e,0.0,0.000000e+00
EX_cpd00099_e,0.0,0.000000e+00
EX_cpd01080_e,0.0,0.000000e+00
SK_cpd11416_c,0.0,0.000000e+00


In [6]:
add_pfba(universal)

In [7]:
rxns_to_remove = [rxn for rxn in universal.reactions if rxn.id in [rxn.id for rxn in testmod_removed.reactions]]
universal.remove_reactions(rxns_to_remove)
universal.add_reactions([rxn for rxn in testmod_removed.reactions])

In [8]:
universal.reactions.get_by_id('bio1').lower_bound = 0.05

In [9]:
solution = universal.optimize()

In [10]:
get_fluxes = set([r.id for r in universal.reactions]) - set([r.id for r in rxns_to_remove])
{rxn:solution.x_dict[rxn] for rxn in get_fluxes if abs(solution.x_dict[rxn]) > 1E-8}

{'EX_cpd00007_e': 0.000396982929734022,
 'EX_cpd00009_e': 0.0012064020421086823,
 'EX_cpd00010_e': -0.0007939658594680549,
 'EX_cpd00011_e': -3.214498279977362,
 'EX_cpd00028_e': -0.0007939658594680431,
 'EX_cpd00030_e': -0.00039698292973402153,
 'EX_cpd00034_e': -0.00039698292973402153,
 'EX_cpd00035_e': 2.83974479696018,
 'EX_cpd00039_e': -0.016275074099941857,
 'EX_cpd00041_e': 0.05333011897452703,
 'EX_cpd00048_e': -0.00039698292973402153,
 'EX_cpd00051_e': -0.07783263332980914,
 'EX_cpd00054_e': 0.8319268284849711,
 'EX_cpd00058_e': -0.0003969829297340022,
 'EX_cpd00060_e': -0.009316579548261813,
 'EX_cpd00063_e': -0.00039698292973402153,
 'EX_cpd00064_e': 0.07783263332980914,
 'EX_cpd00066_e': -0.0088094324187051,
 'EX_cpd00069_e': -0.00555199112899785,
 'EX_cpd00080_e': -0.060138403383065434,
 'EX_cpd00092_e': 2.5738945086084772,
 'EX_cpd00098_e': -0.7243878117332512,
 'EX_cpd00099_e': -0.00039698292973402153,
 'EX_cpd00119_e': -0.004097410427304695,
 'EX_cpd00122_e': 0.66810658

In [11]:
# Combine the universal and testmod
universal.reactions.get_by_id('bio1')

Reaction identifier,bio1
Name,GramPositiveBiomass auto biomass
Memory address,0x07f41ca6933c8
Stoichiometry,41.2913947104178 cpd00002_c + 0.00793965859468043 cpd00003_c + 0.00793965859468043 cpd00006_c + 0.00793965859468043 cpd00010_c + 0.00793965859468043 cpd00015_c + 0.00793965859468043 cpd00016_c + 0.... 41.2913947104178 ATP + 0.00793965859468043 NAD + 0.00793965859468043 NADP + 0.00793965859468043 CoA + 0.00793965859468043 FAD + 0.00793965859468043 Pyridoxal phosphate + 0.00793965859468043 S-Adeno...
GPR,
Lower bound,0.05
Upper bound,1000.0


In [12]:
# verify that the model produces biomass
testmod_removed.optimize()

,fluxes,reduced_costs
rxn05667_c,0.0,-0.000000e+00
rxn00322_c,0.0,0.000000e+00
rxn05457_c,0.0,-0.000000e+00
rxn04271_c,0.0,-0.000000e+00
rxn13783_c,0.0,0.000000e+00
...,...,...
EX_cpd00066_e,0.0,0.000000e+00
EX_cpd00099_e,0.0,0.000000e+00
EX_cpd01080_e,0.0,0.000000e+00
SK_cpd11416_c,0.0,0.000000e+00


In [13]:
testmod_removed_restored = testmod_removed.copy()
testmod_removed_restored.add_reactions([universal.reactions.get_by_id('rxn03408_c'),\
                                       universal.reactions.get_by_id('rxn05440_c'),\
                                       universal.reactions.get_by_id('rxn23650_c')])
testmod_removed_restored.optimize()

,fluxes,reduced_costs
rxn05667_c,-1000.000000,-6.215068e-03
rxn00322_c,0.000000,-6.505213e-19
rxn05457_c,0.000000,0.000000e+00
rxn04271_c,0.000000,0.000000e+00
rxn13783_c,93.459087,0.000000e+00
...,...,...
SK_cpd11416_c,93.459087,0.000000e+00
bio1,93.459087,-9.107705e-16
rxn03408_c,0.168793,1.387779e-17
rxn05440_c,7.770704,3.469447e-18


In [14]:
reaction_variables = ((rxn.forward_variable, rxn.reverse_variable)
                          for rxn in universal.reactions)

In [15]:
variables = chain(*reaction_variables)

In [16]:
universal.reactions[0].forward_variable

0.0 <= rxn31485_c <= 1000.0

In [17]:
universal.objective.get_linear_coefficients(variables)

{0 <= rxn10531_c <= 1000.0: 1.0,
 0.0 <= rxn34290_c <= 1000.0: 1.0,
 0 <= rxn27395_c_reverse_0aa9a <= 0: 1.0,
 0 <= rxn01401_c_reverse_5231f <= 1000.0: 1.0,
 0 <= rxn12042_c <= 1000.0: 1.0,
 0 <= rxn35073_c_reverse_e2f53 <= 1000.0: 1.0,
 0 <= rxn22698_c_reverse_073ae <= 1000.0: 1.0,
 0 <= rxn10966_c_reverse_b9355 <= 1000.0: 1.0,
 0 <= rxn03122_c <= 1000.0: 1.0,
 0 <= rxn15408_c_reverse_e5038 <= 0: 1.0,
 0 <= rxn11914_c <= 1000.0: 1.0,
 0.0 <= rxn18274_c <= 1000.0: 1.0,
 0 <= rxn02814_c_reverse_549aa <= 1000.0: 1.0,
 0 <= rxn09605_c <= 1000.0: 1.0,
 0 <= rxn06857_c <= 1000.0: 1.0,
 0.0 <= rxn18587_c <= 1000.0: 1.0,
 0 <= rxn05877_c <= 1000.0: 1.0,
 0.0 <= rxn34886_c <= 1000.0: 1.0,
 0.0 <= rxn34104_c <= 1000.0: 1.0,
 0 <= rxn17942_c_reverse_ee858 <= 0: 1.0,
 0 <= rxn23992_c_reverse_d3990 <= 0: 1.0,
 0 <= rxn25520_c_reverse_4923d <= 0: 1.0,
 0.0 <= rxn22248_c <= 1000.0: 1.0,
 0 <= rxn10708_c_reverse_db418 <= 1000.0: 1.0,
 0 <= rxn28767_c_reverse_7b992 <= 1000.0: 1.0,
 0 <= rxn10760_c <= 

In [18]:
# load all models, perform FVA, and determine which metabolites are essential in the majority of all models
species_to_essential = {}
for modelfile in os.listdir('../data/modelseed_models/'):
    species_name = modelfile.split('/')[-1].split('.')[0]
    print("performing FVA for " + species_name)
    model = cobra.io.load_json_model('../data/modelseed_models/'+modelfile)
    exchanges = model.exchanges
    for rxn in exchanges:
        rxn.lower_bound = -1000
    fva_result = flux_variability_analysis(model=model,fraction_of_optimum=0.01,reaction_list=exchanges)
    essentials = list(fva_result.loc[fva_result['maximum'] < -0.0001].index)
    species_to_essential[species_name] = essentials

performing FVA for Neisseria gonorrhoeae
performing FVA for Bacillus megaterium
performing FVA for Streptococcus mitis
performing FVA for Corynebacterium glutamicum
performing FVA for Enterococcus faecalis
performing FVA for Streptococcus equinus
performing FVA for Bacillus pumilus
performing FVA for Corynebacterium efficiens
performing FVA for Flavobacterium johnsoniae
performing FVA for Stenotrophomonas maltophilia
performing FVA for Haemophilus influenzae
performing FVA for Pseudomonas stutzeri
performing FVA for Neisseria flavescens
performing FVA for Staphylococcus aureus
performing FVA for Streptococcus oralis
performing FVA for Haemophilus parasuis
performing FVA for Staphylococcus epidermidis
performing FVA for Salmonella enterica subsp
performing FVA for Streptococcus gallolyticus
performing FVA for Chryseobacterium gleum
performing FVA for Bacillus subtilis subsp
performing FVA for Gordonia bronchialis
performing FVA for Achromobacter xylosoxidans
performing FVA for Listeria 

In [19]:
count_essential = {}
for species in species_to_essential.keys():
    essentials = species_to_essential[species]
    for metabolite in essentials:
        if metabolite in count_essential.keys():
            count_essential[metabolite] +=1
        else:
            count_essential[metabolite] = 1

In [20]:
majority_essential = [exchange for exchange in count_essential.keys() if \
                      count_essential[exchange] > len(species_to_essential.keys())/5.0]
majority_essential_as_compounds = [x[3:] for x in majority_essential]
for met in majority_essential_as_compounds:
    if met not in biolog_base_dict.keys():
        biolog_base_dict[met] = 1000

In [21]:
biolog_base_dict

{'cpd00001_e': 1000,
 'cpd00007_e': 1000,
 'cpd00009_e': 1000,
 'cpd00010_e': 1000,
 'cpd00011_e': 1000,
 'cpd00012_e': 1000,
 'cpd00013_e': 1000,
 'cpd00028_e': 1000,
 'cpd00030_e': 1000,
 'cpd00034_e': 1000,
 'cpd00039_e': 1000,
 'cpd00048_e': 1000,
 'cpd00051_e': 1000,
 'cpd00058_e': 1000,
 'cpd00060_e': 1000,
 'cpd00063_e': 1000,
 'cpd00065_e': 1000,
 'cpd00066_e': 1000,
 'cpd00067_e': 1000,
 'cpd00069_e': 1000,
 'cpd00080_e': 1000,
 'cpd00099_e': 1000,
 'cpd00111_e': 1000,
 'cpd00119_e': 1000,
 'cpd00149_e': 1000,
 'cpd00205_e': 1000,
 'cpd00220_e': 1000,
 'cpd00254_e': 1000,
 'cpd00264_e': 1000,
 'cpd00268_e': 1000,
 'cpd00305_e': 1000,
 'cpd00322_e': 1000,
 'cpd00355_e': 1000,
 'cpd00393_e': 1000,
 'cpd00516_e': 1000,
 'cpd00971_e': 1000,
 'cpd01080_e': 1000,
 'cpd03696_e': 1000,
 'cpd03847_e': 1000,
 'cpd10515_e': 1000,
 'cpd10516_e': 1000,
 'cpd11606_e': 1000}

In [22]:
test_mod_pheno = biolog_thresholded.loc['Staphylococcus aureus']
test_mod_pheno = list(test_mod_pheno[test_mod_pheno == True].index)

In [23]:
# check for biolog base components in the model
add_mets = []
add_exchanges = []
for met in list(biolog_base_dict.keys()):
    try:
        testmod.metabolites.get_by_id(met)
    except:
        print('no '+met)
        add_met = universal.metabolites.get_by_id(met).copy()
        add_mets.append(add_met)

testmod.add_metabolites(add_mets)

for met in list(biolog_base_dict.keys()):
    # Search for exchange reactions
    try:
        testmod.reactions.get_by_id('EX_'+met)
    except:
        add_met = testmod.metabolites.get_by_id(met)
        ex_rxn = cobra.core.Reaction('EX_' + met)
        ex_rxn.name = "Exchange reaction for " + met
        ex_rxn.lower_bound = -1000
        ex_rxn.upper_bound = 1000
        ex_rxn.add_metabolites({add_met:-1})
        add_exchanges.append(ex_rxn)

    

testmod.add_reactions(add_exchanges)

no cpd11606_e
no cpd03847_e
no cpd00065_e
no cpd00111_e
no cpd00013_e
no cpd00516_e


In [24]:
solution = universal.optimize()
solution.x_dict['rxn31485_c']

0.0

In [25]:
# Grab 5 carbon sources and generate media dictionaries for them
missing_mets = []
missing_exchanges = []
media_dicts = {}
for met_id in test_mod_pheno:
    try:
        testmod.metabolites.get_by_id(met_id)
    except:

        met = universal.metabolites.get_by_id(met_id).copy()
        missing_mets.append(met)
        ex_rxn = cobra.core.Reaction('EX_' + met_id)
        ex_rxn.name = "Exchange reaction for " + met_id
        ex_rxn.lower_bound = -1000
        ex_rxn.upper_bound = 1000
        ex_rxn.add_metabolites({met:-1})
        missing_exchanges.append(ex_rxn)
    media_dicts[met_id] = biolog_base_dict.copy()
    media_dicts[met_id] = {'EX_'+k:v for k,v in media_dicts[met_id].items()}
    media_dicts[met_id]['EX_'+met_id] = 1000
testmod.add_metabolites(missing_mets)
testmod.add_reactions(missing_exchanges)


In [26]:
testmod.solver = 'glpk'


In [27]:
transporters_in_universal = [rxn for rxn in universal.reactions if len(rxn.compartments)>1]


In [28]:
transporters_in_universal = [rxn for rxn in universal.reactions if len(rxn.compartments)>1]
for met in media_dicts.keys():
    metabolite = testmod.metabolites.get_by_id(met)
    base_met_id = met.split('_')[0]
    rxns_with_metabolite = metabolite.reactions
    transport = False
    for rxn in rxns_with_metabolite:
        metabolites = [met_in_rxn.id for met_in_rxn in rxn.metabolites]
        if (base_met_id+'_e' in metabolites and base_met_id+'_c' in metabolites):
            transport = True
    
    pick_transporter = {}
    if not transport:
        print("missing transporter for " + metabolite.name)
        for rxn in transporters_in_universal:
            metabolites = [met_in_rxn.id for met_in_rxn in rxn.metabolites]
            if (base_met_id+'_e' in metabolites and base_met_id+'_c' in metabolites):
                pick_transporter[met] = rxn.id
                print("found transporter for "+metabolite.name)
                print("transporter is " + str(rxn.reversibility))

missing transporter for D-Lyxitol
found transporter for D-Lyxitol
transporter is True
found transporter for D-Lyxitol
transporter is True
found transporter for D-Lyxitol
transporter is False
missing transporter for Sucrose
found transporter for Sucrose
transporter is False
found transporter for Sucrose
transporter is True
missing transporter for Dextrin
found transporter for Dextrin
transporter is True
missing transporter for 2-Hydroxybutyrate
found transporter for 2-Hydroxybutyrate
transporter is True
missing transporter for D-glucose-6-phosphate
found transporter for D-glucose-6-phosphate
transporter is True
found transporter for D-glucose-6-phosphate
transporter is True
found transporter for D-glucose-6-phosphate
transporter is True
missing transporter for N-Acetyl-D-mannosamine
found transporter for N-Acetyl-D-mannosamine
transporter is True
missing transporter for LACT
found transporter for LACT
transporter is False
found transporter for LACT
transporter is True
missing transporte

In [29]:
# Add the transporters to the model
transporters_to_add = list(pick_transporter.values())
transporter_list = []
for rxn in transporters_to_add:
    transporter_list.append(universal.reactions.get_by_id(rxn).copy())
testmod.add_reactions(transporter_list)

# remove the added transporters from the universal model
universal.remove_reactions([universal.reactions.get_by_id(rxn) for rxn in transporters_to_add])

In [30]:
optcomplete = testmod.optimize().f

In [31]:
for rxn in testmod.exchanges:
    rxn.lower_bound = -1000
optcomplete = testmod.optimize().f

In [32]:
fva_result = flux_variability_analysis(model=testmod,fraction_of_optimum=0.05,reaction_list=testmod.exchanges)

In [33]:
essentials = list(fva_result.loc[fva_result['maximum'] < -0.00001].index)
for rxn in essentials:
    mets = testmod.reactions.get_by_id(rxn).metabolites
    for met in mets:
        print(met.id,met.name)
#[[met.name for met in mets] for mets in rxn.metabolites for rxn in essentials]

cpd00058_e Cu2+
cpd10515_e Fe2+
cpd00179_e Maltose
cpd00060_e L-Methionine
cpd00034_e Zn2+
cpd10516_e fe3
cpd03696_e Ursin
cpd00277_e Deoxyguanosine
cpd00220_e Riboflavin
cpd00028_e Heme
cpd00254_e Mg
cpd00205_e K+
cpd00039_e L-Lysine
cpd00030_e Mn2+
cpd00149_e Co2+
cpd00305_e Thiamin
cpd00264_e Spermidine
cpd00069_e L-Tyrosine
cpd00268_e H2S2O3
cpd00063_e Ca2+
cpd00393_e Folate
cpd00355_e Nicotinamide ribonucleotide
cpd00048_e Sulfate
cpd00119_e L-Histidine
cpd00066_e L-Phenylalanine
cpd00099_e Cl-
cpd01080_e ocdca


In [34]:
biolog_base_composition

,Name,ID
0,H2O,cpd00001_e
1,O2,cpd00007_e
2,Phosphate,cpd00009_e
3,CO2,cpd00011_e
4,NH3,cpd00013_e
5,Mn2+,cpd00030_e
6,Zn2+,cpd00034_e
7,Sulfate,cpd00048_e
8,Cu2+,cpd00058_e
9,Ca2+,cpd00063_e


In [35]:
sources = list(media_dicts.keys())
sub_dict = {sources[0]:media_dicts[sources[0]],
           sources[1]:media_dicts[sources[1]],
           sources[2]:media_dicts[sources[2]],
           sources[3]:media_dicts[sources[3]],
           sources[4]:media_dicts[sources[4]]}

In [ ]:
iterative_gapfill_from_binary_phenotypes(testmod,universal,sub_dict,5,\
                                         lower_bound=0.05,\
                                         inclusion_threshold=1E-8,\
                                         exchange_reactions=False,\
                                         demand_reactions=False)

In [ ]:
sub_dict

In [ ]:
modexchanges = [rxn.id for rxn in testmod.exchanges]
for reaction in missing_exchanges:
    if not reaction.id in modexchanges:
        print(reaction)

In [ ]:
# Make sure transporters exist for all metabolites that enabled growth

In [ ]:
#Bacillus megaterium.json